In [ ]:
# notebook for new ircc dataset (combined_event_log_anonymous.csv)

# TODO
# √ rule out infrequent variants _per subprocess_

# mark subprocesses that are fully automated (put in separate diagram??)

# in general, nesting subprocesses in dcr (paper)

In [ ]:
# data/combined_event_log_anonymous.csv
# -> data/combined_event_log-filt_evt1p.csv
#   -> data/combined_event_log-filt_evt1p-time1m.csv

# subprocesses:
#   data/combined_event_log-filt_evt1p-time1m.csv
#   -> data/combined_event_log-abstracted.csv + sublogs in level2 (based on status) 
#       -> data/combined_event_log-abstracted2.csv + sublogs in level1 (based on nesting)

In [ ]:
import pm4py
import pandas as pd
import numpy as np
from mine_utils import get_log

# pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 200

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Preprocess logs

## Variant analysis

In [ ]:
# huge amount of variability

# total # traces = 7734 traces

# (non-abstr) new log: # vars = 7665, ratio = 99.10783553141971
#   (a variant covers at most 4 traces ...)
#   filter evts in less than X of cases:
#       1%: 7664 variants: minus 1 variant
#       10%: 7657 variants: minus 8 variants
#   same time for events within 1min:
#       ** 6030 variants: minus 1635 variants
#       + above (1%): 6014 variants: minus 1651 variants
#       + above (10%): 5841 variants: minus 1824 variants

# abstr new log: # vars = 5145, ratio = 66.5244375484872
#   same time for events within 1min: # vars = 3654, ratio = 47.25
#   + filter evts in less than 1% of cases: # vars = 3564, ratio = 46.08

# (abstr or log: # traces = 7734, # vars = 2642, ratio = 34.16084820274114)


In [ ]:
from variant_stats import get_variants_stats, get_variant_ratio, get_variant_coverage, get_covering_variants, filter_traces_on_variants

In [ ]:
log = pd.read_csv("data/combined_event_log-filt_evt1p.csv")
vars_log = get_variants_stats(log)
print(get_variant_ratio(log, vars_log))
vars_log

In [ ]:
# much bigger improvement, clearly

log = pd.read_csv("data/combined_event_log-time1m.csv")
vars_log = get_variants_stats(log)
print(get_variant_ratio(log, vars_log))
vars_log

In [ ]:
log = pd.read_csv("data/combined_event_log-filt_evt1p-time1m.csv")
vars_log = get_variants_stats(log)
print(get_variant_ratio(log, vars_log))
vars_log

In [ ]:
log = pd.read_csv("data/combined_event_log_anonymous.csv")
log['act_upd_date'] = pd.to_datetime(log['act_upd_date'])
log = log.rename({ 'app_num': 'case:concept:name', 'activity_full_value': 'concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)

abstr_log_or = pd.read_csv("data/or/abstract_log-starts_ends-v2.csv")
abstr_log_new = pd.read_csv("data/combined_event_log-abstracted2.csv")

In [ ]:
print("total # traces:", len(log['case:concept:name'].unique()))

vars_log = get_variants_stats(log)
vars_abstr_log_or = get_variants_stats(abstr_log_or)
vars_abstr_log_new = get_variants_stats(abstr_log_new)

print("log:", get_variant_ratio(log, vars_log))
print("abstr log or:", get_variant_ratio(log, vars_abstr_log_or))
print("abstr log new:", get_variant_ratio(log, vars_abstr_log_new))

## Infrequent events

In [ ]:
log = pd.read_csv("data/combined_event_log_anonymous.csv")
log = log.rename({ 'app_num': 'case:concept:name', 'activity_full_value': 'concept:name' }, axis=1)

In [ ]:
log[log['activity']=='Other Reqs Assessment']['activity_status'].drop_duplicates()

In [ ]:
from log_stats import count_cases_per_event

activ_cases_counts = count_cases_per_event('concept:name', 'case:concept:name', log).reset_index()

In [ ]:
activ_cases_counts[activ_cases_counts['perc']<1] #.to_csv("data/dropped_events.csv")

In [ ]:
activ_cases_counts[activ_cases_counts['perc']<10]

In [ ]:
# let's filter on activities that occur in 1% or less of cases
to_drop = activ_cases_counts.loc[activ_cases_counts['perc']<1, 'concept:name']
log_filter = log[~ log['concept:name'].isin(to_drop)]

In [ ]:
log_filter.to_csv("data/combined_event_log-filt_evt1p.csv")

## Timestamp differences

In [ ]:
# log = pd.read_csv("data/combined_event_log_anonymous.csv")

log = pd.read_csv("data/combined_event_log-filt_evt1p.csv")
log = log.drop('Unnamed: 0', axis=1)

log = log.rename({ 'app_num': 'case:concept:name', 'activity_full_value': 'concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])

In [ ]:
from mine_utils import get_time_diff, equal_timestamps_interval

In [ ]:
log = get_time_diff(log)

In [ ]:
log[log['activity']=='Associate Medicals'].groupby('case:concept:name')['time_diff'].mean().mean()

In [ ]:
# (check all sequential events with exact same timestamp)
print("num simult (0 sec):", len(log[log['time_diff']==0]), " <> total num:", log.shape[0])

In [ ]:
# (check all sequential events with less than 1min difference in timestamps)
print("num simult (1 min):", len(log[log['time_diff'] < 60]), " <> total num:", log.shape[0])

In [ ]:
# (takes ca. 2-4 sec)

log = equal_timestamps_interval(log, 60) # 60 sec

In [ ]:
# log.to_csv("data/combined_event_log-time1m.csv")
log.to_csv("data/combined_event_log-filt_evt1p-time1m.csv")

# checkout variant analysis for seeing whether it improved matters

# Subprocesses

## Subprocesses based on status

In [36]:
# log = pd.read_csv("data/combined_event_log_anonymous.csv")
# log = log.rename({ 'app_num': 'case:concept:name', 'act_upd_date': 'time:timestamp' }, axis=1)

log = pd.read_csv("data/combined_event_log-filt_evt1p-time1m.csv")
log = log.drop('Unnamed: 0', axis=1)
log

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_12934/2661719508.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv("data/combined_event_log-filt_evt1p-time1m.csv")


,index,case:concept:name,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name,time:timestamp
0,0,0,0,1-SQ1RI5F,NaN,Application Status,Prospective,Vietnam,Vietnam,Application Status - Prospective,2024-06-03 09:20:37.327
1,5,0,2,1-SQ1RI5F,NaN,Biometric Assessment,Required,Vietnam,Vietnam,Biometric Assessment - Required,2024-06-03 09:20:37.327
2,10,0,3,1-SQ1RI7W,NaN,Biometrics - RCMP,Not Started,Vietnam,Vietnam,Biometrics - RCMP - Not Started,2024-06-03 09:20:37.327
3,11,0,4,1-SQ1190E,NaN,Biometrics Fee,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
4,15,0,4,1-SQ1190E,NaN,Biometrics Fee,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
...,...,...,...,...,...,...,...,...,...,...,...
541091,541090,7733,9,1-STCSFDC,NaN,Verification,Completed,Belgium,Cameroon,Verification - Completed,2024-06-17 07:46:39.197
541092,541092,7733,281,1-STC2A6X,NaN,Eligibility Assessment,Review Required,Belgium,Cameroon,Eligibility Assessment - Review Required,2024-06-26 08:11:25.043
541093,541095,7733,281,1-STC2A6X,NaN,Application Status,Closed,Belgium,Cameroon,Application Status - Closed,2024-06-28 10:45:09.050
541094,541093,7733,281,1-STC2A6X,NaN,Eligibility Assessment,Failed,Belgium,Cameroon,Eligibility Assessment - Failed,2024-06-28 10:45:09.050


In [37]:
# get all unique activities
log['activity'].drop_duplicates().sort_values().to_excel("data/all_activities.xlsx")

In [38]:
# get activity subprocesses
activ_lifecycles = log[['activity', 'activity_status']].drop_duplicates().sort_values(by=['activity'])
activ_lifecycles.to_excel("data/activity_lifecycles.xlsx")

In [39]:
# count number of sub-activities in each subprocess
counts = log[['activity', 'activity_status']].drop_duplicates().groupby('activity')['activity_status'].count()
counts = counts.sort_values(ascending=False)
counts

activity
Biographic                      12
Biometric - FCC - USA           10
Verification                     6
Eligibility Assessment           6
Candidate ID                     6
Medical Assessment               5
Criminality Assessment           5
Criminality                      5
Integrated Search                5
Candidate ID Decision            5
Medical                          5
Biometrics - RCMP                5
Biometric - FCC Detail - USA     5
Auto Promote                     5
Identity Verification            4
Biometric Assessment             4
Candidate Search                 4
Biometric - FCC - NZ             4
Auto Promote Decision            4
Associate Medicals               4
Final Assessment                 4
Other Reqs Assessment            4
Application Status               3
Post Identity Checks             3
Study Permit Fee                 2
Work Permit Fee                  2
Identity Review                  2
Biometrics Fee                   2
Biometrics 

In [40]:
# only separate subprocesses with >= 3 sub-activities
parent_activ = counts[counts >= 3].reset_index()
parent_activ

,activity,activity_status
0,Biographic,12
1,Biometric - FCC - USA,10
2,Verification,6
3,Eligibility Assessment,6
4,Candidate ID,6
5,Medical Assessment,5
6,Criminality Assessment,5
7,Criminality,5
8,Integrated Search,5
9,Candidate ID Decision,5


In [41]:
subproc_evts = log[log['activity'].isin(parent_activ['activity'])]
subproc_evts

,index,case:concept:name,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name,time:timestamp
0,0,0,0,1-SQ1RI5F,NaN,Application Status,Prospective,Vietnam,Vietnam,Application Status - Prospective,2024-06-03 09:20:37.327
1,5,0,2,1-SQ1RI5F,NaN,Biometric Assessment,Required,Vietnam,Vietnam,Biometric Assessment - Required,2024-06-03 09:20:37.327
2,10,0,3,1-SQ1RI7W,NaN,Biometrics - RCMP,Not Started,Vietnam,Vietnam,Biometrics - RCMP - Not Started,2024-06-03 09:20:37.327
7,30,0,5,1-SQ1S9OF,NaN,Candidate ID,In Progress,Vietnam,Vietnam,Candidate ID - In Progress,2024-06-03 09:20:37.327
8,25,0,5,1-SQ1S9OF,NaN,Candidate ID,Not Started,Vietnam,Vietnam,Candidate ID - Not Started,2024-06-03 09:20:37.327
...,...,...,...,...,...,...,...,...,...,...,...
541091,541090,7733,9,1-STCSFDC,NaN,Verification,Completed,Belgium,Cameroon,Verification - Completed,2024-06-17 07:46:39.197
541092,541092,7733,281,1-STC2A6X,NaN,Eligibility Assessment,Review Required,Belgium,Cameroon,Eligibility Assessment - Review Required,2024-06-26 08:11:25.043
541093,541095,7733,281,1-STC2A6X,NaN,Application Status,Closed,Belgium,Cameroon,Application Status - Closed,2024-06-28 10:45:09.050
541094,541093,7733,281,1-STC2A6X,NaN,Eligibility Assessment,Failed,Belgium,Cameroon,Eligibility Assessment - Failed,2024-06-28 10:45:09.050


In [42]:
non_subproc_evts = log[~ log['activity'].isin(parent_activ['activity'])]
non_subproc_evts

,index,case:concept:name,act_upd_by,activity_id,group,activity,activity_status,cor,citz,concept:name,time:timestamp
3,11,0,4,1-SQ1190E,NaN,Biometrics Fee,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
4,15,0,4,1-SQ1190E,NaN,Biometrics Fee,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
5,12,0,4,1-SQ1190E,NaN,Biometrics Fee,Paid,Vietnam,Vietnam,Biometrics Fee - Paid,2024-06-03 09:20:37.327
6,16,0,4,1-SQ1190E,NaN,Biometrics Fee,Paid,Vietnam,Vietnam,Biometrics Fee - Paid,2024-06-03 09:20:37.327
27,3,0,1,0,NaN,Misrepresentation,NIL,Vietnam,Vietnam,Misrepresentation - NIL,2024-06-03 09:20:37.327
...,...,...,...,...,...,...,...,...,...,...,...
541070,541034,7733,1,0,NaN,Received,Date,Belgium,Cameroon,Received - Date,2024-06-06 10:36:57.677
541071,541043,7733,1092,1-STC2A8U,NaN,Study Permit Fee,Outstanding,Belgium,Cameroon,Study Permit Fee - Outstanding,2024-06-06 10:36:57.677
541072,541047,7733,1092,1-STC2A8U,NaN,Study Permit Fee,Outstanding,Belgium,Cameroon,Study Permit Fee - Outstanding,2024-06-06 10:36:57.677
541073,541044,7733,1092,1-STC2A8U,NaN,Study Permit Fee,Paid,Belgium,Cameroon,Study Permit Fee - Paid,2024-06-06 10:36:57.677


### Create separate logs per subprocess

In [43]:
from separ_subproc import separ_subproc

separ_subproc(subproc_evts, non_subproc_evts, 'activity', 'activity_status', 'concept:name', "lifecycles/level2/", "data/combined_event_log-abstracted.csv")

Application Status (# events: 23263)
Associate Medicals (# events: 1097)
Auto Promote (# events: 29528)
Auto Promote Decision (# events: 22250)
Biographic (# events: 30777)
Biometric - FCC - NZ (# events: 2586)
Biometric - FCC - USA (# events: 30328)
Biometric - FCC Detail - USA (# events: 4734)
Biometric Assessment (# events: 14929)
Biometrics - RCMP (# events: 25579)
Candidate ID (# events: 29864)
Candidate ID Decision (# events: 23962)
Candidate Search (# events: 23534)
Criminality (# events: 21929)
Criminality Assessment (# events: 21927)
Eligibility Assessment (# events: 19051)
Final Assessment (# events: 15772)
Identity Verification (# events: 1120)
Integrated Search (# events: 31244)
Medical (# events: 6652)
Medical Assessment (# events: 6636)
Other Reqs Assessment (# events: 20615)
Post Identity Checks (# events: 819)
Verification (# events: 27317)


,case:concept:name,activity,index,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name,time:timestamp
0,0,Application Status,0,0,1-SQ1RI5F,None,Prospective,Vietnam,Vietnam,Application Status [begin],2024-06-03 09:20:37.327
5,0,Biometric Assessment,5,2,1-SQ1RI5F,None,Required,Vietnam,Vietnam,Biometric Assessment [begin],2024-06-03 09:20:37.327
6,0,Biometrics - RCMP,10,3,1-SQ1RI7W,None,Not Started,Vietnam,Vietnam,Biometrics - RCMP [begin],2024-06-03 09:20:37.327
270950,0,Biometrics Fee,11,4,1-SQ1190E,NaN,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
270951,0,Biometrics Fee,15,4,1-SQ1190E,NaN,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
...,...,...,...,...,...,...,...,...,...,...,...
270948,7733,Other Reqs Assessment,541091,281,1-STC2A6X,None,NIL,Belgium,Cameroon,Other Reqs Assessment [end],2024-06-17 07:46:39.197
270949,7733,Verification,541090,9,1-STCSFDC,None,Completed,Belgium,Cameroon,Verification [end],2024-06-17 07:46:39.197
270933,7733,Application Status,541095,281,1-STC2A6X,None,Closed,Belgium,Cameroon,Application Status [end],2024-06-28 10:45:09.050
270945,7733,Eligibility Assessment,541093,281,1-STC2A6X,None,Failed,Belgium,Cameroon,Eligibility Assessment [end],2024-06-28 10:45:09.050


## Subprocesses based on nesting

### Check original nesting file

In [44]:
nestings = pd.read_excel("data/nested_activities-original.xlsx")
log = pd.read_csv("data/combined_event_log_anonymous.csv")

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_12934/1550013220.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv("data/combined_event_log_anonymous.csv")


In [45]:
nestings[~ (nestings['Activity'].isin(log['activity']))]

,Activity,Parent Item
0,A39,Other Req Assessment
7,Biometric - FCC Detail - NZ,Info Sharing Assessment
10,Biometric - FCC Reciprocal - NZ,Info Sharing Assessment
11,Biometric - FCC Reciprocal - USA,Info Sharing Assessment
12,Biometric - RCMP,Criminality Activity
14,Cost Recovery,Biometrics Family Rate Fee
15,Cost Recovery,Biometrics Fee
16,Cost Recovery,Open Work Permit Holder Fee Fee
17,Cost Recovery,Overpayment Fee
18,Cost Recovery,Study Permit Fee


In [46]:
cost_recov = nestings[nestings['Activity']=='Cost Recovery']['Parent Item']
cost_recov[~ (cost_recov.isin(log['activity']))]

Series([], Name: Parent Item, dtype: object)

In [47]:
print(len(log[log['activity']=='Criminality Assessment']))
print(len(log[log['activity']=='Medical Assessment']))
print(len(log[log['activity']=='Misrep Assessment']))
print(len(log[log['activity']=='Security Assessment']))

21927
6675
144
41


In [48]:
print(log[log['activity'].str.startswith('Biographic')]['activity'].drop_duplicates())
print()
print(log[log['activity'].str.startswith('Biometric')]['activity'].drop_duplicates())
print()
print(log[log['activity'].str.startswith('Criminal')]['activity'].drop_duplicates())
print()
print(log[log['activity'].str.startswith('Medical')]['activity'].drop_duplicates())
print()
print(log[log['activity'].str.startswith('Security')]['activity'].drop_duplicates())

48                     Biographic
181248    Biographic - Correction
Name: activity, dtype: object

5                     Biometric Assessment
10                       Biometrics - RCMP
11                          Biometrics Fee
49                   Biometric - FCC - USA
327           Biometric - FCC Detail - USA
1673            Biometrics Family Rate Fee
3739                  Biometric - FCC - NZ
120969               Biometric - FCC - AUS
120976        Biometric - FCC Detail - AUS
382302    Biometric - FCC Reciprocal - AUS
Name: activity, dtype: object

6               Criminality
7    Criminality Assessment
Name: activity, dtype: object

70               Medical
72    Medical Assessment
Name: activity, dtype: object

4904               Security
4905    Security Assessment
Name: activity, dtype: object


### Find subprocesses

In [49]:
nestings = pd.read_excel("data/nested_activities-fixed.xlsx")
nestings

,Activity,Parent Item
0,Associate Medicals,Medical Activity
1,Biographic,Info Sharing Assessment
2,Biographic - Correction,Info Sharing Assessment
3,Biometric - FCC - AUS,Info Sharing Assessment
4,Biometric - FCC - NZ,Info Sharing Assessment
5,Biometric - FCC - USA,Info Sharing Assessment
6,Biometric - FCC Detail - AUS,Info Sharing Assessment
7,Biometric - FCC Detail - USA,Info Sharing Assessment
8,Biometric - FCC Reciprocal - AUS,Info Sharing Assessment
9,Biometric Assessment,Info Sharing Assessment


In [50]:
# make sure there's no non-existent activities in the nesting file
or_log = pd.read_csv("data/combined_event_log_anonymous.csv")
nestings[~ nestings['Activity'].isin(or_log['activity'])].sort_values(by='Activity')

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_12934/4061471006.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  or_log = pd.read_csv("data/combined_event_log_anonymous.csv")


,Activity,Parent Item


In [51]:
abstr_log = pd.read_csv("data/combined_event_log-abstracted.csv")
abstr_log

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_12934/3727316693.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  abstr_log = pd.read_csv("data/combined_event_log-abstracted.csv")


,case:concept:name,activity,index,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name,time:timestamp
0,0,Application Status,0,0,1-SQ1RI5F,NaN,Prospective,Vietnam,Vietnam,Application Status [begin],2024-06-03 09:20:37.327
1,0,Biometric Assessment,5,2,1-SQ1RI5F,NaN,Required,Vietnam,Vietnam,Biometric Assessment [begin],2024-06-03 09:20:37.327
2,0,Biometrics - RCMP,10,3,1-SQ1RI7W,NaN,Not Started,Vietnam,Vietnam,Biometrics - RCMP [begin],2024-06-03 09:20:37.327
3,0,Biometrics Fee,11,4,1-SQ1190E,NaN,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
4,0,Biometrics Fee,15,4,1-SQ1190E,NaN,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327
...,...,...,...,...,...,...,...,...,...,...,...
376528,7733,Other Reqs Assessment,541091,281,1-STC2A6X,NaN,NIL,Belgium,Cameroon,Other Reqs Assessment [end],2024-06-17 07:46:39.197
376529,7733,Verification,541090,9,1-STCSFDC,NaN,Completed,Belgium,Cameroon,Verification [end],2024-06-17 07:46:39.197
376530,7733,Application Status,541095,281,1-STC2A6X,NaN,Closed,Belgium,Cameroon,Application Status [end],2024-06-28 10:45:09.050
376531,7733,Eligibility Assessment,541093,281,1-STC2A6X,NaN,Failed,Belgium,Cameroon,Eligibility Assessment [end],2024-06-28 10:45:09.050


In [52]:
# connect parent items to events
# left merge; also keep events that are not being nested
abstr_log_parent = abstr_log.merge(nestings, left_on='activity', right_on='Activity', how='left')
abstr_log_parent

,case:concept:name,activity,index,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name,time:timestamp,Activity,Parent Item
0,0,Application Status,0,0,1-SQ1RI5F,NaN,Prospective,Vietnam,Vietnam,Application Status [begin],2024-06-03 09:20:37.327,NaN,NaN
1,0,Biometric Assessment,5,2,1-SQ1RI5F,NaN,Required,Vietnam,Vietnam,Biometric Assessment [begin],2024-06-03 09:20:37.327,Biometric Assessment,Info Sharing Assessment
2,0,Biometrics - RCMP,10,3,1-SQ1RI7W,NaN,Not Started,Vietnam,Vietnam,Biometrics - RCMP [begin],2024-06-03 09:20:37.327,Biometrics - RCMP,Criminality Activity
3,0,Biometrics Fee,11,4,1-SQ1190E,NaN,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327,Biometrics Fee,Cost Recovery
4,0,Biometrics Fee,15,4,1-SQ1190E,NaN,Outstanding,Vietnam,Vietnam,Biometrics Fee - Outstanding,2024-06-03 09:20:37.327,Biometrics Fee,Cost Recovery
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376528,7733,Other Reqs Assessment,541091,281,1-STC2A6X,NaN,NIL,Belgium,Cameroon,Other Reqs Assessment [end],2024-06-17 07:46:39.197,Other Reqs Assessment,Other Req Activity
376529,7733,Verification,541090,9,1-STCSFDC,NaN,Completed,Belgium,Cameroon,Verification [end],2024-06-17 07:46:39.197,Verification,Other Req Activity
376530,7733,Application Status,541095,281,1-STC2A6X,NaN,Closed,Belgium,Cameroon,Application Status [end],2024-06-28 10:45:09.050,NaN,NaN
376531,7733,Eligibility Assessment,541093,281,1-STC2A6X,NaN,Failed,Belgium,Cameroon,Eligibility Assessment [end],2024-06-28 10:45:09.050,NaN,NaN


In [53]:
subproc_evts = abstr_log_parent[abstr_log_parent['Parent Item'].notna()]
# (non-nested events; those not merged with parent)
non_subproc_evts = abstr_log_parent[abstr_log_parent['Parent Item'].isna()]

### Create separate logs per nested activity

In [54]:
from separ_subproc import separ_subproc

separ_subproc(subproc_evts, non_subproc_evts, 'Parent Item', 'concept:name', 'concept:name', "lifecycles/level1", "data/combined_event_log-abstracted2.csv")

Cost Recovery (# events: 82369)
Criminality Activity (# events: 43613)
Info Sharing Assessment (# events: 50046)
Medical Activity (# events: 12728)
Other Req Activity (# events: 27988)


,case:concept:name,Parent Item,activity,index,act_upd_by,activity_id,group,activity_status,cor,citz,concept:name,time:timestamp,Activity
65294,0,NaN,Application Status,0,0,1-SQ1RI5F,NaN,Prospective,Vietnam,Vietnam,Application Status [begin],2024-06-03 09:20:37.327,NaN
65295,0,NaN,Candidate ID Decision,27,6,1-SQ1JBR7,NaN,In Progress,Vietnam,Vietnam,Candidate ID Decision [begin],2024-06-03 09:20:37.327,NaN
65296,0,NaN,Candidate ID,30,5,1-SQ1S9OF,NaN,In Progress,Vietnam,Vietnam,Candidate ID [begin],2024-06-03 09:20:37.327,NaN
65297,0,NaN,Candidate Search,29,5,1-SQ11912,NaN,Complete,Vietnam,Vietnam,Candidate Search [begin],2024-06-03 09:20:37.327,NaN
65298,0,NaN,Candidate Search,19,5,1-SQ11912,NaN,Submit,Vietnam,Vietnam,Candidate Search [end],2024-06-03 09:20:37.327,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65292,7733,Info Sharing Assessment,Biometric - FCC - USA,541089,18,1-STCSFD6,None,NRT,Belgium,Cameroon,Info Sharing Assessment [end],2024-06-11 23:52:27.260,Biometric - FCC - USA
65293,7733,Other Req Activity,Verification,541090,9,1-STCSFDC,None,Completed,Belgium,Cameroon,Other Req Activity [end],2024-06-17 07:46:39.197,Verification
225080,7733,NaN,Application Status,541095,281,1-STC2A6X,NaN,Closed,Belgium,Cameroon,Application Status [end],2024-06-28 10:45:09.050,NaN
225081,7733,NaN,Eligibility Assessment,541093,281,1-STC2A6X,NaN,Failed,Belgium,Cameroon,Eligibility Assessment [end],2024-06-28 10:45:09.050,NaN


## Sanity check

In [ ]:
import os

dir = "lifecycles/level2/logs"
sublogs_lvl2 = [ (f, pd.read_csv(os.path.join(dir, f))) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) ]

### Check original events

In [ ]:
# same input as subprocesses part
# (NOTE - original log will have to be extended with index for sanity code to work)
log = pd.read_csv("data/combined_event_log-filt_evt1p-time1m.csv")

In [ ]:
from collections import defaultdict

found_indexes = set()

# > subprocesses based on status

# - find original events in the sublogs

for f, sublog in sublogs_lvl2:    
    f_name = f[0:f.index(".")]
    
    # join original log with the sublog
    log_merged = log.merge(sublog, left_on='index', right_on='index')
    # print(log_merged)
    
    # (apply same string operation as on the file name)
    activ_file = (log_merged['activity_x'].str.replace("/", "_") == f_name)
    # for all matches, activity names should correspond to file name
    assert (activ_file).all(), f_name + " <> " + log_merged[~ activ_file]['activity_x']
    # for all matches, activity statuses should correspond to sublog activity name
    assert (log_merged['activity_status'] == log_merged['concept:name_y']).all(), "file: {f_name}"
    
    found_indexes.update(log_merged['index'])

In [ ]:
# - find original events in the abstracted log

# only interested in activities without parent (non-nested)
log_filter = log[~ log['activity'].isin(parent_activ['activity'])]
# join original log with the abstracted log
log_merged = log_filter.merge(abstr_log, left_on='index', right_on='index')
# print(log_merged)

activ_name = (log_merged['concept:name_x'] == log_merged['concept:name_y'])
# activity names should correspond
assert (activ_name).all(), log_merged[~ activ_name][['concept:name_x', 'concept:name_y']]

found_indexes.update(log_merged['index'])

In [ ]:
# all events found?
assert len(found_indexes) == log.shape[0], f"{len(found_indexes)} <> {log.shape[0]}"

### Check subprocesses

In [ ]:
# > subprocesses based on nesting

# per case, check if activity's start/end markers correspond to first & last sorted events of the activity

def check_subproc(abstr_log, marker_label, is_start):
    # group all start/end markers by activity
    groups = abstr_log[abstr_log['concept:name'].str.endswith(marker_label)].groupby('activity')

    # for each activity & their markers for all cases
    for activ, g in groups:
        # sort on case
        g = g.sort_values(by='case:concept:name').reset_index()
        
        # find sublog corresponding to activity
        for i in [ 1, 2 ]:
            path = f"lifecycles/level2/logs/{activ.replace('/', '_')}.csv"
            if not os.path.exists(path):
                continue
            sublog = pd.read_csv(path)
        
        # print(activ, path)
        
        # first, in the activity's sublog, find the firsts/lasts for each case
        gb = sublog.groupby('case:concept:name')
        delims = (gb.first() if is_start else gb.last()).reset_index()
        # also sort on case
        delims = delims.sort_values(by='case:concept:name').reset_index()
        
        # firsts/lasts should be the same as the start/end markers
        assert(g['index'] == delims['index']).all(), activ
        
check_subproc(abstr_log, " [begin]", True)
check_subproc(abstr_log, " [end]", False)

path = "lifecycles/level1/logs/"

from pathlib import Path
paths = Path(path).rglob("*.csv")
for path in paths:
    abstr_sublog = pd.read_csv(path)
    check_subproc(abstr_sublog, " [begin]", True)
    check_subproc(abstr_sublog, " [end]", False)
    

## Variant analysis (bis)

### Main variant (no filtering)

In [ ]:
from variant_stats import get_variants_stats, get_variant_ratio, get_variant_coverage, get_covering_variants, filter_traces_on_variants

In [ ]:
log = get_log("data/combined_event_log-abstracted2.csv")
log = log[['case:concept:name', 'concept:name', 'time:timestamp']]

In [ ]:
var_stats = get_variants_stats(log)
print(get_variant_ratio(log, var_stats))
var_stats

In [ ]:
singleton_vars = var_stats[var_stats['cov_amt']==1]

# ooph, a lot of traces here ...
singleton_vars['cov_amt'].sum() / var_stats['cov_amt'].sum()

### Subprocess variants (filtering!)

In [ ]:
# some ad-hoc analysis

# log = pd.read_csv("/Users/wvw/git/pm/ircc/lifecycles/level1/logs/Other Req Activity.csv")
# var_stats = get_variants_stats(log, plot=False)
# print(var_stats)
# rare_vars = var_stats[var_stats['cov_perc'] < 1]
# print()
# print(rare_vars['cov_perc'].sum().round(2))
# print((rare_vars['sequence'].count() / var_stats.shape[0] * 100).round(2))

In [ ]:
# filter the variants
# (ca. 3-5 s)

dir = "/Users/wvw/git/pm/ircc/lifecycles"
for subdir in [ "level1", "level2" ]:
    print(">", subdir)
    for path in Path(os.path.join(dir, subdir, "logs")).rglob("*.csv"):
        name = os.path.basename(path)
        log = pd.read_csv(path)
        var_stats = get_variants_stats(log, plot=False)
        
        rare_vars = var_stats[var_stats['cov_perc'] < 1]
        print(name, ":", rare_vars['cov_perc'].sum().round(2), "% traces", "(", (rare_vars['sequence'].count() / var_stats.shape[0] * 100).round(2), "% vars" ")")
        
        flog = filter_traces_on_variants(log, var_stats[var_stats['cov_perc'] >= 1])
        flog.to_csv(path)

    print()

## (Timestamp differences (bis))

In [ ]:
log = get_log("data/combined_event_log-abstracted2.csv")
log = log[['case:concept:name', 'concept:name', 'time:timestamp']]
log

In [ ]:
from mine_utils import get_time_diff

logd = get_time_diff(log)

In [ ]:
# (check all sequential events with less than 1min difference in timestamps)
print("num simult (1 min):", len(logd[ (logd['time_diff'] > 0) & (logd['time_diff'] < 60)]), " <> total num:", logd.shape[0])

# none to be found

## (Infrequent events (bis))

In [ ]:
log = pd.read_csv("data/combined_event_log-abstracted2.csv")

from log_stats import count_cases_per_event
activ_cases_counts = count_cases_per_event('concept:name', 'case:concept:name', log).reset_index()

In [ ]:
activ_cases_counts[activ_cases_counts['perc']<1]

# Mine process models

## Ad-hoc mining

In [ ]:
from variant_stats import get_variants_stats, get_variant_ratio, get_variant_coverage, get_covering_variants, filter_traces_on_variants
from mine_utils import get_log, ProcAnn, mine_heur, mine_induct, mine_alpha, mine_dfg

### Main process

In [ ]:
log = get_log("data/combined_event_log-abstracted2.csv")

cost_recov = log[(log['concept:name']=='Cost Recovery [begin]') | (log['concept:name']=='Cost Recovery [end]')]

log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
cost_recov.to_csv("data/cost_recov.csv")
xes_export.apply(cost_recov, "data/cost_recov.xes")

mine_heur(log, ProcAnn.FREQ, "graphs/combined_event_log-abstracted2-time1m")
mine_induct(log, convert_to='petri_net', output_path="graphs/combined_event_log-abstracted2-time1m-pn")

log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
xes_export.apply(log, "data/combined_event_log-abstracted2.xes")

### Subprocess

In [ ]:
verif = get_log("/Users/wvw/git/pm/ircc/lifecycles/level2/logs/Verification.csv")
verif

In [ ]:
vars = get_variants_stats(verif)
vars

In [ ]:
mine_dfg(verif)

## Systematic mining

In [55]:
from mine_utils import ProcAnn, mine_heur, mine_induct, mine_alpha, mine_dfg
import pm4py.objects.log.exporter.xes.exporter as xes_export
from pathlib import Path

In [56]:
import shutil, os

def init_subdir(subdir, subsubdirs=[]):
    if os.path.exists(subdir):
        shutil.rmtree(subdir)
    os.mkdir(subdir)
    for subsubdir in subsubdirs:
        os.mkdir(os.path.join(subdir, subsubdir))

In [57]:
induct_formats = ['bpmn', 'petri_net']
formats_with_ann = ['dfg']

In [58]:
def save_entries_json(names, default_format, default_ann, path):
    def entry_pref():
        if default_format in formats_with_ann:
            return f"{{ \"format\": \"{default_format}\", \"ann\": \"{default_ann.value}\" }}"
        else:
            return f"{{ \"format\": \"{default_format}\" }}"
    
    all = "[" + ", ".join(map(lambda n: f"\"{n}\"", names)) + "]"
    prefs = "{" + ", ".join(map(lambda n: f"\"{n}\": {entry_pref()}", names)) + "}"
    obj = f"{{ \"all\": {all}, \"prefs\": {prefs} }}"
    open(os.path.join(path, "graphs.json"), "w").write(obj)

In [59]:
# mine process models for sublogs
# (ca. 1 min)

for lvl in [ 1, 2 ]:
    # subprocess level (levels 1-2)
    subdir = f"lifecycles/level{lvl}"

    # default model & annotation to be shown
    default_format = "bpmn"
    default_ann = ProcAnn.FREQ

    init_subdir(os.path.join(subdir, "xes"))
    init_subdir(os.path.join(subdir, "dfg"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
    init_subdir(os.path.join(subdir, "heur"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
    for format in induct_formats:
        init_subdir(os.path.join(subdir, format))

    names = [ ]
    for path in Path(os.path.join(subdir, "logs")).rglob("*.csv"):
        file = os.path.basename(path)
        name = file[0: file.index(".csv")]
        print(name)
        names.append(name)
        
        log = pd.read_csv(path)
        log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
        
        log['case:concept:name'] = log['case:concept:name'].astype('int64')
        log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])
        
        xes_export.apply(log, os.path.join(subdir, "xes", name + ".xes"))
        
        for ann in ProcAnn:
            mine_dfg(log, ann, output_path=os.path.join(subdir, "dfg", ann.value, name), save_gviz=True)
            mine_heur(log, ann, output_path=os.path.join(subdir, "heur", ann.value, name), save_gviz=True)
        
        for format in induct_formats:
            mine_induct(log, convert_to=format, output_path=os.path.join(subdir, format, name), save_gviz=True)

    save_entries_json(names, default_format, default_ann, subdir)

Medical Activity


exporting log, completed traces ::   0%|          | 0/3163 [00:00<?, ?it/s]

Other Req Activity


exporting log, completed traces ::   0%|          | 0/6780 [00:00<?, ?it/s]

Info Sharing Assessment


...

exporting log, completed traces ::   0%|          | 0/3042 [00:00<?, ?it/s]

Biometrics - RCMP


exporting log, completed traces ::   0%|          | 0/7236 [00:00<?, ?it/s]

Application Status


exporting log, completed traces ::   0%|          | 0/7734 [00:00<?, ?it/s]

In [60]:
# mine models for main process
# (ca. 30-40 sec)

log = pd.read_csv("data/combined_event_log-abstracted2.csv")
log = log[['case:concept:name', 'time:timestamp', 'concept:name']]
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])

subdir = "lifecycles/level0"

default_format = "dcr"
default_ann = ProcAnn.FREQ

init_subdir(os.path.join(subdir, "logs"))
init_subdir(os.path.join(subdir, "dfg"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
init_subdir(os.path.join(subdir, "heur"), [ ProcAnn.FREQ.value, ProcAnn.PERF.value ])
init_subdir(os.path.join(subdir, "bpmn"))
init_subdir(os.path.join(subdir, "petri_net"))
init_subdir(os.path.join(subdir, "xes"))

name = "main"
log.to_csv(os.path.join(subdir, "logs", name + ".csv"))

for ann in ProcAnn:
    mine_dfg(log, ann, output_path=os.path.join(subdir, "dfg", ann.value, name), save_gviz=True)
    mine_heur(log, ann, output_path=os.path.join(subdir, "heur", ann.value, name), save_gviz=True)

for format in induct_formats:
    mine_induct(log, convert_to=format, output_path=os.path.join(subdir, format, name), save_gviz=True)

xes_export.apply(log, os.path.join(subdir, "xes", name + ".xes"))

save_entries_json([name], default_format, default_ann, subdir)

exporting log, completed traces ::   0%|          | 0/7734 [00:00<?, ?it/s]

In [61]:
# mine DCR: see
# /Users/wvw/git/pm/declarative/dcr4py/pm4py-dcr/ircc_dcr.ipynb